# WZE-UAV Image Classification using Deep Learning

In [1]:
import os
import gc
import glob
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

In [2]:
import torch
import torchvision.transforms as T
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from mlxtend.plotting import plot_confusion_matrix

In [3]:
import wze_uav.data_loader as data_loader
import wze_uav.models as models
from wze_uav.engine_Copy1 import *
from wze_uav.utils2 import *
from wze_uav.log_writer import create_writer
from wze_uav.datasplit import *
from efficientnet import model_effnet #for custom effnet with n_channels input
import wandb

#### Get PyTorch version

In [4]:
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 1.13.1+cu116
torchvision version: 0.14.1+cu116


#### Preparing device agnostic code 

In [5]:
# ensure device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# get index of currently selected device
print(f"Index of current divice: {torch.cuda.current_device()}")
# get number of GPUs available
print(f"Number of GPUs available: {torch.cuda.device_count()}")
# get the name of the device
print(f"GPU Model: {torch.cuda.get_device_name(0)}")

cuda
Index of current divice: 0
Number of GPUs available: 1
GPU Model: Quadro RTX 8000


#### Ensure reproducibility 

In [6]:
# for more information, see also: https://pytorch.org/docs/stable/notes/randomness.html

# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)
    # seed for numpy
    np.random.seed(seed)

set_seeds(42) 

# Set to true -> might speed up the process but should be set to False if reproducible results are desired
torch.backends.cudnn.benchmark = False


#### Define file directory

In [7]:
#####################################################################################
# 3 channel input (r-g-b)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb"

# 4 channel input (r-g-b-nir)
data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb-nir"

# 5 channel input (r-g-b-re-nir)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb-re-nir"
#####################################################################################

#### Get all file paths

In [8]:
fn_list = os.listdir(data_path)
path_list = []
path_list_2020 = []
path_list_2021 = []
path_list_2022 = []
# Iterate over all datafiles
for year in fn_list:
    year_dir = f'{data_path}\\{year}'
    for filename in os.listdir(year_dir):
        path = f'{year_dir}\\{filename}'
        path_list.append(path)
        if year == '2020':
            path == f'{year_dir}\\{filename}'
            path_list_2020.append(path)
        elif year == '2021':
            path == f'{year_dir}\\{filename}'
            path_list_2021.append(path)
        else:
            path == f'{year_dir}\\{filename}'
            path_list_2022.append(path)

In [9]:
path_list

['D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10527_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10533_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10545_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10547_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr10912_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11032_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11040_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11044_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_findatree\\Datasplit\\ROI\\rgb-nir\\2020\\tnr11048_rois.hdf5',
 'D:\\Drohnendaten\\10_WZE-UAV\\Auswertung_fin

#### Create unique hash IDs for every individual tree

In [10]:
hashID_dict = data_loader.get_unique_treeID_v2(path_list)

Creating unique tree IDs...:   0%|          | 0/647 [00:00<?, ?it/s]

#### Import all imagery, labels and other features from hdf5 files

In [540]:
image_set, label_set, species_set, kkl_set, bk_set, hash_id = data_loader.hdf5_to_img_label_v3(path_list,
                                                                                               hashID_dict,
                                                                                               load_sets=["images_masked"])

Processing hdf5 datasets:   0%|          | 0/647 [00:00<?, ?it/s]

#### Convert nbv to classes

In [541]:
label_set = nbv_to_sst_3classes(label_set)

In [542]:
unique_values = np.unique(hash_id[:,0])
num_unique = len(unique_values)
print(f"There are {num_unique} unique values within hash_id.")

# Split unique hash_id values into train and test sets
#train_ids, test_ids = train_test_split(unique_values, test_size=test_size, random_state=random_state)
print(unique_values)

There are 7087 unique values within hash_id.
['0019c16a' '0023a9ed' '002bd7e9' ... 'ffed9046' 'fffa193b' 'fffef24f']


In [543]:
# get image IDs for each unique hash ID and year
image_ids_2020 = hash_id[hash_id[:, 1] == '2020'][:, 0]
image_ids_2021 = hash_id[hash_id[:, 1] == '2021'][:, 0]
image_ids_2022 = hash_id[hash_id[:, 1] == '2022'][:, 0]

In [544]:
# split unique hash IDs into train and test sets
train_ids, test_ids = train_test_split(unique_values, test_size=0.1667, random_state=42)

In [545]:
len(train_ids)
len(test_ids)

1182

In [546]:
# create boolean arrays indicating which indices belong to train or test sets
train_indices = np.isin(hash_id[:, 0], train_ids)
len(train_indices)

num_true = np.count_nonzero(train_indices)
num_false = np.count_nonzero(~train_indices)

print("Number of True in train_indices:", num_true)
print("Number of False in train_indices:", num_false)

Number of True in train_indices: 15659
Number of False in train_indices: 3146


In [559]:
np.random.seed(2) # set the seed value

test_indices = np.zeros_like(train_indices)  # initialize to all False
for hash_id_test in test_ids:
    # select one image ID randomly from either 2020 or 2021 or 2022 for each unique hash ID in the test set
    year = np.random.choice(['2020', '2021', '2022'])
    image_ids = hash_id[(hash_id[:, 0] == hash_id_test) & (hash_id[:, 1] == year), 0]
    
    # mark the index corresponding to the selected image ID and hash ID as True in the test indices array
    test_indices[(hash_id[:, 0] == hash_id_test) & (hash_id[:, 1] == year) & (np.isin(hash_id[:, 0], image_ids))] = True

print("Train indices:", train_indices)
print("Test indices:", test_indices)


Train indices: [[ True]
 [ True]
 [ True]
 ...
 [ True]
 [ True]
 [ True]]
Test indices: [[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [560]:
num_true = np.count_nonzero(test_indices)
num_false = np.count_nonzero(~test_indices)
print("Number of True in test_indices:", num_true)
print("Number of False in test_indices:", num_false)


Number of True in test_indices: 1056
Number of False in test_indices: 17749


In [561]:


print("Number of True in train_indices:", num_true)
print("Number of False in train_indices:", num_false)

Number of True in train_indices: 1056
Number of False in train_indices: 17749


In [562]:
# Reshape boolean arrays to match shape of image_set and label_set
train_indices = train_indices.reshape(-1, 1)
test_indices = test_indices.reshape(-1, 1)

num_true = np.count_nonzero(test_indices)
num_false = np.count_nonzero(~test_indices)

print("Number of True in test_indices:", num_true)
print("Number of False in test_indices:", num_false)
print(train_indices.shape)

Number of True in test_indices: 1056
Number of False in test_indices: 17749
(18805, 1)


In [563]:
# Select images and labels for train and test sets
train_val_image_set = image_set[train_indices[:, 0]]
train_val_label_set = label_set[train_indices[:, 0]]
train_val_hash_id = hash_id[train_indices[:, 0]]

train_val_species_set = species_set[train_indices[:, 0]]

test_image_set = image_set[test_indices[:, 0]]
test_label_set = label_set[test_indices[:, 0]]
test_hash_id = hash_id[test_indices[:, 0]]
test_species_set = species_set[test_indices[:, 0]]

train_val_label_set = train_val_label_set.reshape(-1, 1)
test_label_set = test_label_set.reshape(-1, 1)
#train_hash_id = train_hash_id.reshape(-1, 1)
#test_hash_id = test_hash_id.reshape(-1, 1)
train_val_species_set = train_val_species_set.reshape(-1, 1)
test_species_set = test_species_set.reshape(-1, 1)
#print(train_val_hash_id.shape)
print(train_val_image_set.shape)
print(train_val_label_set.shape)

(15659, 250, 250, 4)
(15659, 1)


In [564]:
print("Check shapes:\n")
print(f"Images train dataset: {train_val_image_set.shape}")
print(f"Labels train dataset: {train_val_label_set.shape}\n")

print(f"Images test dataset: {test_image_set.shape}")
print(f"Labels test dataset: {test_label_set.shape}\n")
print('-'*50)
#print (f"Check if the split was stratified: (random_state={random_state})")
print(f"Healthy trees in train dataset: {np.count_nonzero(train_val_label_set == 0)}")
print(f"Stressed trees in train dataset: {np.count_nonzero(train_val_label_set == 1)}")
print(f"Dead trees in train dataset: {np.count_nonzero(train_val_label_set == 2)}")
print(f"Healthy trees in test dataset: {np.count_nonzero(test_label_set == 0)}")
print(f"Stressed trees in test dataset: {np.count_nonzero(test_label_set == 1)}")
print(f"Dead trees in test dataset: {np.count_nonzero(test_label_set == 2)}")
print(f"Ratio health trees in test dataset: {np.count_nonzero(test_label_set == 0)/np.count_nonzero(train_val_label_set == 0)}")
print(f"Ratio stressed trees in test dataset: {np.count_nonzero(test_label_set == 1)/np.count_nonzero(train_val_label_set == 1)}")
print(f"Ratio dead trees in test dataset: {np.count_nonzero(test_label_set == 2)/np.count_nonzero(train_val_label_set == 2)}")

Check shapes:

Images train dataset: (15659, 250, 250, 4)
Labels train dataset: (15659, 1)

Images test dataset: (1056, 250, 250, 4)
Labels test dataset: (1056, 1)

--------------------------------------------------
Healthy trees in train dataset: 14175
Stressed trees in train dataset: 1223
Dead trees in train dataset: 261
Healthy trees in test dataset: 960
Stressed trees in test dataset: 81
Dead trees in test dataset: 15
Ratio health trees in test dataset: 0.06772486772486773
Ratio stressed trees in test dataset: 0.06623058053965658
Ratio dead trees in test dataset: 0.05747126436781609


#### Split data and seperate a test dataset

In [ ]:
image_set, label_set, hash_id, species_set, test_image_set, test_label_set, test_hash_id, test_species_set = data_split(image_set, label_set, hash_id, species_set, test_size=0.15, random_state=42)


(15659,)

#### Check if any hash ID is in both train and test dataset

In [565]:
hash_set = set(train_val_hash_id[:,0].flatten())
test_hash_set = set(test_hash_id[:,0].flatten())
intersection = hash_set.intersection(test_hash_set)
if intersection:
    print(f"Hash_id values in both train and test sets: {intersection}")
else:
    print("There are no hash_id values in both train and test datasets. The datasplit was successful")

There are no hash_id values in both train and test datasets. The datasplit was successful


#### Check feature distribution of the Test dataset

In [566]:
def count_occurrences(data, value):
    count = 0
    for item in data:
        if item == value:
            count += 1
    return count

print("Test dataset")
print(f"Test data healthy trees: {count_occurrences(test_label_set, 0)}")
print(f"Test data stressed trees: {count_occurrences(test_label_set, 1)}")
print(f"Test data dead trees: {count_occurrences(test_label_set, 2)}")
print(f"Test data pine trees: {count_occurrences(test_species_set, 134)}")
print(f"Test data spruces: {count_occurrences(test_species_set, 118)}")
print("-"*50)

print("Remaining dataset")
print(f"Remaining data healthy trees: {count_occurrences(train_val_label_set, 0)}")
print(f"Remaining data stressed trees: {count_occurrences(train_val_label_set, 1)}")
print(f"Remaining data dead trees: {count_occurrences(train_val_label_set, 2)}")
print(f"Remaining data pine trees: {count_occurrences(train_val_species_set, 134)}")
print(f"Remaining data spruces: {count_occurrences(train_val_species_set, 118)}")
print("-"*50)

Test dataset
Test data healthy trees: 960
Test data stressed trees: 81
Test data dead trees: 15
Test data pine trees: 278
Test data spruces: 388
--------------------------------------------------
Remaining dataset
Remaining data healthy trees: 14175
Remaining data stressed trees: 1223
Remaining data dead trees: 261
Remaining data pine trees: 3960
Remaining data spruces: 5833
--------------------------------------------------


In [567]:
# train transform with augmentation. 
transform_train = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(p=0.5), transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomRotation(degrees=[0,360])])

# test and val dataset transform without augmentation. 
transform = transforms.Compose([transforms.ToTensor()])

# class names need to fit the customDataset class used e.g. 3 classes -> use CustomDataset3Classes
#class_names = ['healthy', 'slightly_stressed', 'moderately_stressed', 'highly_stressed', 'dead']
#class_names = ['healthy', 'moderately_stressed', 'highly_stressed', 'dead']
class_names = ['healthy', 'stressed', 'dead']

# set seeds
g = torch.Generator()
g.manual_seed(42)
NUM_WORKERS=3 # should be changed, depending on the system used
batch_size=32

#### Define variables and parameters

In [568]:
print(train_val_image_set.shape)
print(train_val_species_set.shape)
print(train_val_hash_id.shape)
print(test_image_set.shape)

(15659, 250, 250, 4)
(15659, 1)
(15659, 2)
(1056, 250, 250, 4)


In [569]:
# 1. Define number of epochs
epochs = 50
n_bands = image_set[0].shape[2] # get number of bands

# 2. Define model
num_classes = len(class_names)
unfreeze = True # all layer weights get updated
dropout_rate = 0.45 #define dropout rate
model_name = "EffNet_b7_RGB-NIR_3classes_v3_new_val"

# 3. Define loss, optimizer and learning rate scheduler
lr = 0.004 # define learning rate
min_lr = 1e-5 # minimum learning rate threshold
gamma = 0.75 # how fast the learning rate decreases per epoch (low number=faster decrease)
patience = 10

# 4. Create target folder name were to save the tensorboard event files

experiment_name = 'RGB-NIR_3classes_v3_new_val'
extra = "new_val_set"

#----------------------------------------------------------------------
#torch.cuda.empty_cache()
print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes") 

# login to weights & biases to track metrics
wandb.login()
wandb.init(project='wze-uav', entity='simon-ecke')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Memory allocated: 0 bytes


#### Run k-Fold cross-validation on EfficientNet

In [571]:
%%time

# Set the random seeds
set_seeds(42)

# Define the number of folds
num_folds = 5

# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

unique_values = np.unique(train_val_hash_id[:,0]) # now there are less unique values left because we seperated the test dataset
num_unique = len(unique_values)
print(f"There are {num_unique} unique values within hash_id.\n")

# Loop over each fold
for fold, (train_ids, val_ids) in enumerate(kf.split(unique_values)):
    torch.cuda.empty_cache()
    
    # Initialize a new wandb run for this fold
    wandb.init(project='wze-uav', name=f"fold_{fold + 1}_{extra}")
    
    # 1. Split data into train and validation set
    # Get the training and testing data for this fold
    # Use np.isin() to create boolean arrays indicating which indices belong to train or test sets
    train_indices = np.isin(train_val_hash_id[:,0], unique_values[train_ids])
    print(train_indices.shape)
    
    val_indices = np.zeros_like(train_indices)  # initialize to all False
    for hash_id_val in unique_values[val_ids]:
        # select one image ID randomly from either 2020 or 2021 or 2022 for each unique hash ID in the test set
        year = np.random.choice(['2020', '2021', '2022'])
        image_ids = train_val_hash_id[(train_val_hash_id[:,0] == hash_id_val) & (train_val_hash_id[:,1] == year), 0]
    
        # mark the index corresponding to the selected image ID and hash ID as True in the test indices array
        val_indices[(train_val_hash_id[:,0] == hash_id_val) & (train_val_hash_id[:,1] == year) & (np.isin(train_val_hash_id[:,0], image_ids))] = True 
    
    # Reshape boolean arrays to match shape of image_set and label_set
    train_indices = train_indices.reshape(-1, 1)
    val_indices = val_indices.reshape(-1, 1)
    
    # Select images and labels for train and validation sets
    train_image_set = train_val_image_set[train_indices[:, 0]]
    train_label_set = train_val_label_set[train_indices[:, 0]]
    train_hash_id = train_val_hash_id[train_indices[:, 0]][:,0]
    train_species_set = train_val_species_set[train_indices[:, 0]]
    val_image_set = train_val_image_set[val_indices[:, 0]]
    val_label_set = train_val_label_set[val_indices[:, 0]]
    val_hash_id = train_val_hash_id[val_indices[:, 0]][:,0]
    val_species_set = train_val_species_set[val_indices[:, 0]]
    train_label_set = train_label_set.reshape(-1, 1)
    val_label_set = val_label_set.reshape(-1, 1)
    train_species_set = train_species_set.reshape(-1, 1)
    val_species_set = val_species_set.reshape(-1, 1)

         
    print("Check shapes:\n")
    print(f"Images train dataset: {train_image_set.shape}")
    print(f"Labels train dataset: {train_label_set.shape}\n")
    
    print(f"Images validation dataset: {val_image_set.shape}")
    print(f"Labels validation dataset: {val_label_set.shape}\n")
    print('-'*50)
    print (f"Check if the split was stratified: (random_state=42)")
    print(f"Healthy trees in train dataset: {np.count_nonzero(train_label_set == 0)}")
    print(f"Stressed trees in train dataset: {np.count_nonzero(train_label_set == 1)}")
    print(f"Dead trees in train dataset: {np.count_nonzero(train_label_set == 2)}")
    print(f"Healthy trees in validation dataset: {np.count_nonzero(val_label_set == 0)}")
    print(f"Stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)}")
    print(f"Dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)}")
    print(f"Ratio health trees in validation dataset: {np.count_nonzero(val_label_set == 0)/np.count_nonzero(label_set == 0)}")
    print(f"Ratio stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)/np.count_nonzero(label_set == 1)}")
    print(f"Ratio dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)/np.count_nonzero(label_set == 2)}")
    print("-"*50)
   
    # 2. Create train and validation dataset. (choose custom dataset loader with 3 - 5 classes)
    print(f"\nCreating datasets for fold: {fold + 1}\n")
    train_dataset = data_loader.CustomDataset(data=train_image_set, labels=train_label_set, class_names=class_names, species = train_species_set,
                                                         transform=transform_train)
    
    val_dataset = data_loader.CustomDataset(data=val_image_set, labels=val_label_set, class_names=class_names,
                                                       species = val_species_set, transform=transform)
   
    # 3. Create train and validation dataloader
    # create sampler for oversampling of the minority classes
    sampler = data_loader.data_sampler(dataset=train_dataset, class_names=class_names)
    print(f"Creating dataloaders for fold: {fold +1}\n")
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, generator=g,
                              sampler=sampler, shuffle=False, drop_last=True) # shuffle false because of the sampler

    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, shuffle=False,
                             drop_last=True)
    
    model = model_effnet.EfficientNet.from_pretrained('efficientnet-b7', in_channels=n_bands, num_classes=num_classes, dropout_rate=dropout_rate)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    #lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    lr_scheduler = ExponentialLR(optimizer, gamma=gamma)

    fold += 1
    print(f"\n[INFO] Fold number: {fold}")
    print(f"[INFO] Number of epochs: {epochs}")
    print(f"[INFO] Batch_size: {batch_size}")
    print(f"[INFO] Number of bands: {n_bands}")
    print(f"[INFO] Dropout rate: {dropout_rate}")
    print(f"[INFO] Gamma learning rate: {gamma}")
    print(f"[INFO] Memory allocated: {torch.cuda.memory_allocated()} bytes")
    # 4. Train model with k fold dataloaders and track experiments
    
    if fold == 1:
        fold1_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
       
    elif fold == 2:
        fold2_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    elif fold == 3:
        fold3_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    elif fold == 4:
        fold4_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    else:
        fold5_results = train(model=model, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=None, early_stop_patience = patience)
    
    del train_indices, val_indices, train_image_set, train_label_set, train_hash_id, train_species_set, val_image_set, val_label_set, val_hash_id, val_species_set,
    train_dataset, val_dataset, sampler, train_dataloader, val_dataloader, model, loss_fn, optimizer, lr_scheduler
    
    #finish the wandb run
    wandb.finish()
    print("Deleting variables and emptying cache")
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes")
    print("-"*50 + "\n")
    


There are 5905 unique values within hash_id.



(15659,)
Check shapes:

Images train dataset: (12564, 250, 250, 4)
Labels train dataset: (12564, 1)

Images validation dataset: (1043, 250, 250, 4)
Labels validation dataset: (1043, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 11352
Stressed trees in train dataset: 1008
Dead trees in train dataset: 204
Healthy trees in validation dataset: 951
Stressed trees in validation dataset: 71
Dead trees in validation dataset: 21
Ratio health trees in validation dataset: 0.055888575458392105
Ratio stressed trees in validation dataset: 0.04807041299932295
Ratio dead trees in validation dataset: 0.0673076923076923
--------------------------------------------------

Creating datasets for fold: 1

Creating dataloaders for fold: 1

Loaded pretrained weights for efficientnet-b7

[INFO] Fold number: 1
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 4
[INFO] Dropout rate: 0.45
[INFO] G

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.004
Train loss: 0.7491 | Train precision: 0.6398 | Train recall: 0.6018 | Train f1score: 0.6180 | Train acc: 0.6302 | Train kappa: 0.4867 
Val loss: 0.4077 | Val precision: 0.3040 | Val recall: 0.3333 | Val f1score: 0.3180 | Val acc: 0.9111 | Val kappa: 0.0249 

Epoch: 2 
Learning rate: 0.003
Train loss: 0.6358 | Train precision: 0.7203 | Train recall: 0.7127 | Train f1score: 0.7164 | Train acc: 0.6995 | Train kappa: 0.6282 
Val loss: 0.2947 | Val precision: 0.6811 | Val recall: 0.4580 | Val f1score: 0.5109 | Val acc: 0.9043 | Val kappa: 0.3761 

Epoch: 3 
Learning rate: 0.0022500000000000003
Train loss: 0.5826 | Train precision: 0.7527 | Train recall: 0.7530 | Train f1score: 0.7529 | Train acc: 0.7259 | Train kappa: 0.6613 
Val loss: 0.5182 | Val precision: 0.6650 | Val recall: 0.7623 | Val f1score: 0.6611 | Val acc: 0.7568 | Val kappa: 0.3983 

Epoch: 4 
Learning rate: 0.0016875000000000002
Train loss: 0.5740 | Train precision: 0.7647 | Train recall: 0.7587

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
fold,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,█▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train_f1_score,▁▄▅▅▆▆▇▇▇▇▇▇████████
train_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
val_f1_score,▁▅▇▇██████▇█▇████▇█▇
val_loss,▄▁▆█▃▄▄▃▅▃▄▃▄▄▃▄▄▄▄▅
epoch,19
fold,1
learning_rate,2e-05
train_f1_score,0.84773


...

(15659,)
Check shapes:

Images train dataset: (12522, 250, 250, 4)
Labels train dataset: (12522, 1)

Images validation dataset: (1040, 250, 250, 4)
Labels validation dataset: (1040, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 11338
Stressed trees in train dataset: 985
Dead trees in train dataset: 199
Healthy trees in validation dataset: 949
Stressed trees in validation dataset: 70
Dead trees in validation dataset: 21
Ratio health trees in validation dataset: 0.05577103902209685
Ratio stressed trees in validation dataset: 0.04739336492890995
Ratio dead trees in validation dataset: 0.0673076923076923
--------------------------------------------------

Creating datasets for fold: 5

Creating dataloaders for fold: 5

Loaded pretrained weights for efficientnet-b7

[INFO] Fold number: 5
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 4
[INFO] Dropout rate: 0.45
[INFO] Gam

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.004
Train loss: 0.7459 | Train precision: 0.6467 | Train recall: 0.6192 | Train f1score: 0.6314 | Train acc: 0.6289 | Train kappa: 0.4960 
Val loss: 0.3455 | Val precision: 0.4200 | Val recall: 0.3765 | Val f1score: 0.3877 | Val acc: 0.8877 | Val kappa: 0.1396 

Epoch: 2 
Learning rate: 0.003
Train loss: 0.6172 | Train precision: 0.7309 | Train recall: 0.7296 | Train f1score: 0.7302 | Train acc: 0.7091 | Train kappa: 0.6416 
Val loss: 0.3298 | Val precision: 0.6674 | Val recall: 0.6675 | Val f1score: 0.6659 | Val acc: 0.8809 | Val kappa: 0.5452 

Epoch: 3 
Learning rate: 0.0022500000000000003
Train loss: 0.5891 | Train precision: 0.7434 | Train recall: 0.7472 | Train f1score: 0.7453 | Train acc: 0.7191 | Train kappa: 0.6536 
Val loss: 0.4691 | Val precision: 0.7018 | Val recall: 0.7195 | Val f1score: 0.6633 | Val acc: 0.8037 | Val kappa: 0.4505 

Epoch: 4 
Learning rate: 0.0016875000000000002
Train loss: 0.5771 | Train precision: 0.7512 | Train recall: 0.7600

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
fold,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,█▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1_score,▁▄▅▅▆▆▇▇▇▇▇████████████
train_loss,█▅▄▄▃▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
val_f1_score,▁▇▇█▇█▇▇█▇█▇███▇██▇▇▇▇▇
val_loss,▁▁▅▂█▄▆▅▄▆▂▄▃▃▃▃▂▃▃▃▃▃▄
epoch,22
fold,5
learning_rate,1e-05
train_f1_score,0.83031


Deleting variables and emptying cache
Memory allocated: 1055400960 bytes
--------------------------------------------------

CPU times: total: 4h 47min 18s
Wall time: 9h 44min


In [ ]:
# Create a graph to visualize the training and validation loss for each fold
wandb.run.summary["graph"] = wandb.plot.line_series(
    xs="epoch",
    ys=["train_loss", "val_loss"],
    group="fold",
    xaxis="Epoch",
    yaxis="Loss",
    title="Training and Validation Loss by Fold",
)

# Create a graph to visualize the training and validation f1 score for each fold
wandb.run.summary["graph"] = wandb.plot.line_series(
    xs="epoch",
    ys=["train_f1_score", "val_f1_score"],
    group="fold",
    xaxis="Epoch",
    yaxis="Loss",
    title="Training and Validation F1-Score by Fold",
)

In [ ]:
# create test dataset
test_dataset = data_loader.CustomDataset3Classes(
    data = test_image_set,
    labels = test_label_set,
    class_names=class_names, 
    species = test_species_set,
    kkl = test_kkl_set,
    transform=transform
)

# create test dataloader
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             persistent_workers=True,
                             pin_memory=True,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             drop_last=True)

In [ ]:
# Setup the best model filepath
best_model_path = r"C:\Users\lwfeckesim\01_PyTorch\wze-uav\wze-uav-master\effnet_b0\01_18_epochs.pth"

# Instantiate a new instance of EffNetB0 (to load the saved state_dict() to)
unfreeze=True
best_model = models.create_effnetb0(output_shape=num_classes, unfreeze=unfreeze, dropout_rate=dropout_rate, device=device)

# Load the saved best model state_dict()
best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
def make_predictions(model: torch.nn.Module, 
                     test_dataloader: torch.utils.data.DataLoader,
                     device: torch.device):
    # 1. Make predictions with trained model
    y_preds = []
    y_labels = []
    test_loss, test_precision, test_recall, test_f1_score, test_acc = 0, 0, 0, 0, 0
    count = 0
    model.eval()
    with torch.inference_mode():
        for X, y in tqdm(test_dataloader, desc="Making predictions"):
            # Send data and targets to target device
            X, y = X.to(device), y.to(device)
            # Do the forward pass
            y_logit = model(X)
            # Turn predictions from logits -> prediction probabilities -> predictions labels
            y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
            # Put predictions on CPU for evaluation
            y_preds.append(y_pred.cpu())
            y_labels.append(y.cpu())
            
            #other metrics
            test_acc += ((y_pred == y).sum().item()/len(y_pred))
            y_pred_class = y_pred.detach().cpu().numpy() 
            y_class = y.detach().cpu().numpy()
            labels = np.array([0])
            test_precision += precision_score(y_class, y_pred_class, average='macro', zero_division=0, labels=[0,1,2])
            test_recall += recall_score(y_class, y_pred_class, average='macro', zero_division=0, labels=[0,1,2])
            #test_f1_score += f1_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
            
            #if count >= 1:
            #    y_set = torch.cat((y_set, y))
            #    count = count + 1
            #else:
            #    y_set = y
            #    count = count + 1
            
    test_loss = test_loss / len(test_dataloader)
    test_precision = test_precision / len(test_dataloader)
    test_recall = test_recall / len(test_dataloader)
    test_f1_score = test_f1_score / len(test_dataloader)
    #test_kappa = test_kappa / len(dataloader)
    test_acc = test_acc / len(test_dataloader)
    # Concatenate list of predictions into a tensor
    y_pred_tensor = torch.cat(y_preds)
    y_labels_tensor = torch.cat(y_labels)
    test_f1_score = f1_score(y_labels_tensor.detach().cpu().numpy(), y_pred_tensor.cpu().numpy(), average='macro', zero_division=1, labels=[0,1,2])
    
    # Print classification report
    y_true = y_labels_tensor.detach().cpu().numpy()
    report = classification_report(y_true, y_pred_tensor.cpu().numpy(), target_names=class_names)
    print(report)
    
    return y_pred_tensor, y_labels_tensor, test_loss, test_recall, test_precision, test_f1_score, test_acc, y_logit, y_pred, y, y_preds

In [ ]:
# 2. Setup confusion matrix instance and compare predictions to targets
#from wze_uav.analysis import *
y_pred_tensor, y_labels_tensor, test_loss, test_recall, test_precision, test_f1_score, test_acc, y_logit, y_pred, y, y_preds = make_predictions(model=best_model,
                                 test_dataloader=test_dataloader, 
                                 device=device)

y_labels_tensor = y_labels_tensor.detach().cpu().numpy()
y_pred_tensor = y_pred_tensor.detach().cpu().numpy()

#confmat = ConfusionMatrix(num_classes=num_classes, task='multiclass')
#confmat_tensor = confmat(preds=y_pred_tensor,
#                         target=test_labels)
labels = np.array([0,1,2])
confmat = confusion_matrix(y_labels_tensor, y_pred_tensor, labels=labels)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat, # matplotlib likes working with NumPy 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);

print(f"Test loss: {test_loss}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test F1score: {test_f1_score}")
#print(f"Test Kappa: {test_kappa}")
print(f"Test Accuracy: {test_acc}")
print(f"Test Logits: {y_logit}")
print(f"Test Predictions: {y_pred}")
print(f"Test Labels: {y}")

In [ ]:
len(y_preds)

In [ ]:
y_set.cpu()

In [ ]:
test_dataset.labels

In [ ]:
y_preds = []
y_labels = []
labels = np.array([0,1,2])
test_loss, test_precision, test_recall, test_f1_score, test_acc = 0, 0, 0, 0, 0
count = 0
model.eval()
with torch.inference_mode():
    for X, y in tqdm(test_dataloader, desc="Making predictions"):
        # Send data and targets to target device
        X, y = X.to(device), y.to(device)
        # Do the forward pass
        y_logit = model(X)
        # Turn predictions from logits -> prediction probabilities -> predictions labels
        y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
        # Put predictions on CPU for evaluation
        y_preds.append(y_pred.cpu())
        y_labels.append(y.cpu())
        
        #other metrics
        test_acc += ((y_pred == y).sum().item()/len(y_pred))
        y_pred_class = y_pred.detach().cpu().numpy() 
        y_class = y.detach().cpu().numpy()
        test_precision += precision_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        test_recall += recall_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        #test_f1_score += f1_score(y_class, y_pred_class, average='macro', zero_division=1, labels=labels)
        
        #if count >= 1:
        #    y_set = torch.cat((y_set, y))
        #    count = count + 1
        #else:
        #    y_set = y
        #    count = count + 1
        
test_loss = test_loss / len(test_dataloader)
test_precision = test_precision / len(test_dataloader)
test_recall = test_recall / len(test_dataloader)
#test_f1_score = test_f1_score / len(test_dataloader)
#test_kappa = test_kappa / len(dataloader)
test_acc = test_acc / len(test_dataloader)
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)
test_f1_score = f1_score(y_set.detach().cpu().numpy(), y_pred_tensor.cpu().numpy(), average='macro', zero_division=0, labels=[0,1,2])

# Print classification report
y_true = y_set.detach().cpu().numpy()
report = classification_report(y_true, y_pred_tensor.cpu().numpy(), target_names=class_names)
print(report)

In [ ]:
test_f1_score

In [ ]:
make = (y_class == y_pred_class)
make

In [ ]:
torch.softmax(y_logit, dim=1).argmax(dim=1)

In [ ]:
y

In [ ]:
test = (y_pred == y).sum().item()/16

In [ ]:
test

In [ ]:
 y_pred_class = y_pred.detach().cpu().numpy() 

In [ ]:
y_pred_class